In [5]:
import pandas as pd
import numpy as np

In [7]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
train.groupby("Sex")["Survived"].mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

In [13]:
train.groupby("Pclass")["Survived"].mean()

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

In [21]:
#test set copy
predictions = test.copy()
# Predict 1 (survived) if female, 0 (died) if male
predictions["Survived"] = predictions["Sex"].map(lambda x: 1 if x == "female" else 0)
# PassengerId and Survived for submission
submissions = predictions[["PassengerId", "Survived"]]
submissions.to_csv("submission.csv", index=False)

In [29]:
# Step 1: Select features
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

X = train[features].copy()
y = train["Survived"].copy()

# Step 2: Handle missing values
X["Age"] = X["Age"].fillna(X["Age"].median())
X["Embarked"] = X["Embarked"].fillna(X["Embarked"].mode()[0])

# Step 3: Convert categorical text into numbers
X = pd.get_dummies(X)

# Step 4: Prepare the test set in the same way
X_test = test[features].copy()
X_test["Age"] = X_test["Age"].fillna(X_test["Age"].median())
X_test["Fare"] = X_test["Fare"].fillna(X_test["Fare"].median())
X_test["Embarked"] = X_test["Embarked"].fillna(X_test["Embarked"].mode()[0])
X_test = pd.get_dummies(X_test)

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split data into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate on validation set
y_pred = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_pred))


Validation Accuracy: 0.8100558659217877


In [33]:
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate
y_pred_rf = rf_model.predict(X_val)
print("Random Forest Validation Accuracy:", accuracy_score(y_val, y_pred_rf))


Random Forest Validation Accuracy: 0.8044692737430168


In [35]:
# Create a new feature: family size
X["FamilySize"] = X["SibSp"] + X["Parch"] + 1  # +1 to include the passenger themselves

# Rerun the split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Retrain Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_val)

print("Random Forest with FamilySize Accuracy:", accuracy_score(y_val, y_pred_rf))


Random Forest with FamilySize Accuracy: 0.8212290502793296


In [37]:
# Extract Title from Name
train["Title"] = train["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)
test["Title"] = test["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)

# Simplify rare titles into a single category
rare_titles = ["Dr", "Col", "Major", "Rev", "Jonkheer", "Capt", "Don", "Sir", "the Countess", "Lady"]
train["Title"] = train["Title"].replace(rare_titles, "Rare")
test["Title"] = test["Title"].replace(rare_titles, "Rare")

# Group similar ones
train["Title"] = train["Title"].replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})
test["Title"] = test["Title"].replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})

# Add Title into features
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "FamilySize", "Title"]

X = train[features].copy()
y = train["Survived"].copy()

# Fill missing values again (now including Title)
X["Age"] = X["Age"].fillna(X["Age"].median())
X["Embarked"] = X["Embarked"].fillna(X["Embarked"].mode()[0])
X = pd.get_dummies(X)

X_test = test[features].copy()
X_test["Age"] = X_test["Age"].fillna(X_test["Age"].median())
X_test["Fare"] = X_test["Fare"].fillna(X_test["Fare"].median())
X_test["Embarked"] = X_test["Embarked"].fillna(X_test["Embarked"].mode()[0])
X_test = pd.get_dummies(X_test)


<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
C:\Users\faizs\AppData\Local\Temp\ipykernel_9500\4240152083.py:2: SyntaxWarning: invalid escape sequence '\.'
  train["Title"] = train["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)
C:\Users\faizs\AppData\Local\Temp\ipykernel_9500\4240152083.py:3: SyntaxWarning: invalid escape sequence '\.'
  test["Title"] = test["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)
C:\Users\faizs\AppData\Local\Temp\ipykernel_9500\4240152083.py:2: SyntaxWarning: invalid escape sequence '\.'
  train["Title"] = train["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)
C:\Users\faizs\AppData\Local\Temp\ipykernel_9500\4240152083.py:3: SyntaxWarning: invalid escape sequence '\.'
  test["Title"] = test["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)


KeyError: "['FamilySize'] not in index"

In [39]:
# Make sure FamilySize is created in both train and test
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1
test["FamilySize"] = test["SibSp"] + test["Parch"] + 1

# Extract Title (with raw string r"...")
train["Title"] = train["Name"].str.extract(r" ([A-Za-z]+)\.", expand=False)
test["Title"] = test["Name"].str.extract(r" ([A-Za-z]+)\.", expand=False)

# Simplify rare titles
rare_titles = ["Dr", "Col", "Major", "Rev", "Jonkheer", "Capt", "Don", "Sir", "the Countess", "Lady"]
train["Title"] = train["Title"].replace(rare_titles, "Rare")
test["Title"] = test["Title"].replace(rare_titles, "Rare")

# Merge similar titles
train["Title"] = train["Title"].replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})
test["Title"] = test["Title"].replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})

# Define features (now including both FamilySize and Title)
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "FamilySize", "Title"]

# Build X and y
X = train[features].copy()
y = train["Survived"].copy()

# Handle missing values
X["Age"] = X["Age"].fillna(X["Age"].median())
X["Embarked"] = X["Embarked"].fillna(X["Embarked"].mode()[0])
X = pd.get_dummies(X)

X_test = test[features].copy()
X_test["Age"] = X_test["Age"].fillna(X_test["Age"].median())
X_test["Fare"] = X_test["Fare"].fillna(X_test["Fare"].median())
X_test["Embarked"] = X_test["Embarked"].fillna(X_test["Embarked"].mode()[0])
X_test = pd.get_dummies(X_test)


In [41]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Align train/test columns after get_dummies
X_test = X_test.reindex(columns = X.columns, fill_value=0)

# Split (same as before)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Baseline stronger RF (just to check quickly)
rf_baseline = RandomForestClassifier(n_estimators=200, random_state=42)
rf_baseline.fit(X_train, y_train)
pred_baseline = rf_baseline.predict(X_val)
print("Baseline RF (200 trees) Validation Accuracy:", accuracy_score(y_val, pred_baseline))

# Small Grid Search for better hyperparameters (lightweight)
param_grid = {
    "n_estimators": [100, 200, 500],
    "max_depth": [4, 6, 8, None],
    "min_samples_split": [2, 5]
}
rf = RandomForestClassifier(random_state=42)

grid = GridSearchCV(rf, param_grid, cv=5, scoring="accuracy", n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

print("\nGrid search best cross-val score: {:.4f}".format(grid.best_score_))
print("Best params:", grid.best_params_)

# Evaluate best model on validation set
best_rf = grid.best_estimator_
y_pred = best_rf.predict(X_val)
print("Best RF Validation Accuracy:", accuracy_score(y_val, y_pred))

# Feature importances (top 10)
importances = best_rf.feature_importances_
feat_names = X.columns
imp_df = list(zip(feat_names, importances))
imp_df = sorted(imp_df, key=lambda x: x[1], reverse=True)[:10]
print("\nTop 10 feature importances:")
for fname, imp in imp_df:
    print(f"{fname:20s}  {imp:.4f}")


Baseline RF (200 trees) Validation Accuracy: 0.8212290502793296
Fitting 5 folds for each of 24 candidates, totalling 120 fits

Grid search best cross-val score: 0.8370
Best params: {'max_depth': 6, 'min_samples_split': 5, 'n_estimators': 200}
Best RF Validation Accuracy: 0.8324022346368715

Top 10 feature importances:
Title_Mr              0.1830
Sex_female            0.1411
Sex_male              0.1238
Fare                  0.1114
Pclass                0.1027
Age                   0.0701
FamilySize            0.0671
Title_Mrs             0.0514
SibSp                 0.0377
Title_Miss            0.0350


In [43]:
# Predict survival on the test set
test_predictions = best_rf.predict(X_test)

# Create submission DataFrame
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": test_predictions
})

# Save as CSV (ready to upload to Kaggle)
submission.to_csv("faiz_submission.csv", index=False)

print("submission.csv created! You can now upload it to Kaggle.")


submission.csv created! You can now upload it to Kaggle.


In [45]:
train["IsAlone"] = (train["FamilySize"] == 1).astype(int)
test["IsAlone"] = (test["FamilySize"] == 1).astype(int)


In [47]:
train["Deck"] = train["Cabin"].str[0].fillna("U")  # U = Unknown
test["Deck"] = test["Cabin"].str[0].fillna("U")


In [49]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", 
            "FamilySize", "Title", "IsAlone", "Deck"]

# Recreate X and X_test with these features
X = train[features].copy()
y = train["Survived"].copy()

X["Age"] = X["Age"].fillna(X["Age"].median())
X["Fare"] = X["Fare"].fillna(X["Fare"].median())
X["Embarked"] = X["Embarked"].fillna(X["Embarked"].mode()[0])
X = pd.get_dummies(X)

X_test = test[features].copy()
X_test["Age"] = X_test["Age"].fillna(X_test["Age"].median())
X_test["Fare"] = X_test["Fare"].fillna(X_test["Fare"].median())
X_test["Embarked"] = X_test["Embarked"].fillna(X_test["Embarked"].mode()[0])
X_test = pd.get_dummies(X_test)
X_test = X_test.reindex(columns=X.columns, fill_value=0)


In [53]:
rf_model = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=42)
rf_model.fit(X, y)

# Predict on test set
predictions = rf_model.predict(X_test)

# Create final submission
submission = pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": predictions})
submission.to_csv("faiz_submission.csv", index=False)
print("Improved submission.csv created!")


Improved submission.csv created!
